<a href="https://colab.research.google.com/github/adenurchalisa/Automatic-Photo-Clustering-System-Optimization-HDBSCAN/blob/main/src/feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall onnxruntime -y -q
!pip install onnxruntime-gpu -q

!pip install insightface hdbscan -q
!pip install pillow-heif -q

print("install complete guyss")

install complete guyss


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pillow_heif
pillow_heif.register_heif_opener()

In [ ]:
# Face Recognition
import insightface
from insightface.app import FaceAnalysis

In [ ]:
# Clustering
import hdbscan

In [ ]:
# Evaluation & Visualization
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Google Colab
from google.colab import drive

In [ ]:
print("✅ All libraries imported successfully!")
print(f"   - InsightFace version: {insightface.__version__}")
print(f"   - HDBSCAN: installed ✓")
print(f"   - HEIC Support: enabled ✓")

✅ All libraries imported successfully!
   - InsightFace version: 0.7.3
   - HDBSCAN: installed ✓
   - HEIC Support: enabled ✓


In [ ]:
drive.mount('/content/drive')
print("sukses drivenya bro")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
sukses drivenya bro


In [ ]:
DATASET_PATH = "/content/drive/MyDrive/OTW S.KOM/DOKUMENTASI OUTDOOR 2025"

OUTPUT_DIR = "/content/drive/MyDrive/OTW S.KOM/Embeddings"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.webp', '.heic', '.heif'}

In [ ]:
CONFIG = {
    'det_size': (640, 640),  # Detection size untuk InsightFace
    'embedding_dim': 512,     # Dimensi embedding InsightFace
}

In [ ]:
print(f"✅ Configuration set!")
print(f"   - Dataset path: {DATASET_PATH}")
print(f"   - Output directory: {OUTPUT_DIR}")
print(f"   - Supported formats: {IMAGE_EXTENSIONS}")

✅ Configuration set!
   - Dataset path: /content/drive/MyDrive/OTW S.KOM/DOKUMENTASI OUTDOOR 2025
   - Output directory: /content/drive/MyDrive/OTW S.KOM/Embeddings
   - Supported formats: {'.bmp', '.heic', '.heif', '.png', '.jpg', '.jpeg', '.webp'}


In [ ]:
def collect_image_paths(root_dir, extensions=IMAGE_EXTENSIONS):
    image_paths = []

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            # Cek apakah file adalah gambar
            ext = os.path.splitext(filename)[1].lower()
            if ext in extensions:
                full_path = os.path.join(dirpath, filename)
                image_paths.append(full_path)

    # Sort untuk konsistensi
    image_paths.sort()

    return image_paths


# Test: Kumpulkan semua path foto
image_paths = collect_image_paths(DATASET_PATH)

print(f"✅ Found {len(image_paths)} images in dataset")
print(f"\n📁 Sample paths (first 5):")
for path in image_paths[:5]:
    print(f"   - {path}")

✅ Found 2533 images in dataset

📁 Sample paths (first 5):
   - /content/drive/MyDrive/OTW S.KOM/DOKUMENTASI OUTDOOR 2025/Games/DSCF0307.JPG
   - /content/drive/MyDrive/OTW S.KOM/DOKUMENTASI OUTDOOR 2025/Games/DSCF0309.JPG
   - /content/drive/MyDrive/OTW S.KOM/DOKUMENTASI OUTDOOR 2025/Games/DSCF0310.JPG
   - /content/drive/MyDrive/OTW S.KOM/DOKUMENTASI OUTDOOR 2025/Games/DSCF0311.JPG
   - /content/drive/MyDrive/OTW S.KOM/DOKUMENTASI OUTDOOR 2025/Games/DSCF0312.JPG


In [ ]:
from collections import Counter

def analyze_folder_structure(image_paths):
        # Ambil nama folder dari setiap path
    folders = []
    for path in image_paths:
        # Ambil parent folder
        parent = os.path.dirname(path)
        # Ambil nama folder terakhir saja
        folder_name = os.path.basename(parent)
        folders.append(folder_name)

    # Hitung distribusi
    folder_counts = Counter(folders)

    print("📊 DISTRIBUSI FOTO PER FOLDER")
    print("=" * 50)
    for folder, count in folder_counts.most_common():
        print(f"   {folder}: {count} foto")
    print("=" * 50)
    print(f"   TOTAL: {len(image_paths)} foto")

    return folder_counts

# Jalankan analisis
folder_distribution = analyze_folder_structure(image_paths)

📊 DISTRIBUSI FOTO PER FOLDER
   Peserta: 991 foto
   Pengurus & Panitia: 843 foto
   Games: 303 foto
   Materi: 222 foto
   Senior: 129 foto
   Lokasi: 43 foto
   DOKUMENTASI OUTDOOR 2025: 2 foto
   TOTAL: 2533 foto


In [ ]:
def initialize_face_analyzer(det_size=(640, 640)):

    app = FaceAnalysis(
        name='buffalo_l',  # Model yang digunakan
        providers=['CUDAExecutionProvider', 'CPUExecutionProvider']
    )
    app.prepare(ctx_id=0, det_size=det_size)

    print("✅ FaceAnalysis initialized!")
    print(f"   - Detection size: {det_size}")
    print(f"   - Models loaded: {list(app.models.keys())}")

    return app


def extract_features(image_paths, face_app, save_path=None):

    embeddings_list = []
    metadata_list = []

    # Statistics
    stats = {
        'total_images': len(image_paths),
        'images_with_faces': 0,
        'images_no_face': 0,
        'images_multi_face': 0,
        'images_error': 0,
        'total_faces': 0
    }

    # Track images tanpa wajah dan error untuk logging
    no_face_images = []
    error_images = []

    print(f"🔄 Extracting features from {len(image_paths)} images...")
    print("=" * 60)

    for idx, img_path in enumerate(tqdm(image_paths, desc="Processing")):
        try:
            # Load image
            img = np.array(Image.open(img_path).convert('RGB'))

            # Detect faces
            faces = face_app.get(img)

            if len(faces) == 0:
                # Tidak ada wajah terdeteksi
                stats['images_no_face'] += 1
                no_face_images.append(img_path)
                continue

            if len(faces) > 1:
                stats['images_multi_face'] += 1

            stats['images_with_faces'] += 1

            # Ekstrak embedding dari setiap wajah
            for face_idx, face in enumerate(faces):
                embedding = face.embedding  # 512-dimensional

                embeddings_list.append(embedding)
                metadata_list.append({
                    'image_path': img_path,
                    'image_filename': os.path.basename(img_path),
                    'face_index': face_idx,
                    'total_faces_in_image': len(faces),
                    'bbox': face.bbox.tolist(),  # Bounding box
                    'det_score': float(face.det_score)  # Detection confidence
                })

                stats['total_faces'] += 1

        except Exception as e:
            stats['images_error'] += 1
            error_images.append({'path': img_path, 'error': str(e)})
            continue

    # Convert to numpy array
    embeddings = np.array(embeddings_list)

    # Compile results
    results = {
        'embeddings': embeddings,
        'metadata': metadata_list,
        'stats': stats,
        'no_face_images': no_face_images,
        'error_images': error_images
    }

    # Print summary
    print("\n" + "=" * 60)
    print("📊 EXTRACTION SUMMARY")
    print("=" * 60)
    print(f"   Total images processed:    {stats['total_images']}")
    print(f"   Images with faces:         {stats['images_with_faces']}")
    print(f"   Images without faces:      {stats['images_no_face']}")
    print(f"   Images with multiple faces:{stats['images_multi_face']}")
    print(f"   Images with errors:        {stats['images_error']}")
    print(f"   ─────────────────────────────────")
    print(f"   Total faces extracted:     {stats['total_faces']}")
    print(f"   Embedding shape:           {embeddings.shape}")

    # Save results if path provided
    if save_path:
        with open(save_path, 'wb') as f:
            pickle.dump(results, f)
        print(f"\n💾 Results saved to: {save_path}")

    return results

In [ ]:
# Initialize Face Analyzer
face_app = initialize_face_analyzer(det_size=CONFIG['det_size'])

# Define save path
EMBEDDINGS_SAVE_PATH = os.path.join(OUTPUT_DIR, "embeddings_data.pkl")

# Run extraction
# ⚠️ Ini akan memakan waktu tergantung jumlah foto (estimasi: 1-2 detik per foto)
extraction_results = extract_features(
    image_paths=image_paths,
    face_app=face_app,
    save_path=EMBEDDINGS_SAVE_PATH
)

# Simpan embeddings dan metadata ke variabel terpisah untuk kemudahan
embeddings = extraction_results['embeddings']
metadata = extraction_results['metadata']
stats = extraction_results['stats']

print(f"\n✅ Feature extraction complete!")
print(f"   Embeddings shape: {embeddings.shape}")

Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with o

Processing: 100%|██████████| 2533/2533 [42:14<00:00,  1.00s/it]


📊 EXTRACTION SUMMARY
   Total images processed:    2533
   Images with faces:         2365
   Images without faces:      168
   Images with multiple faces:1760
   Images with errors:        0
   ─────────────────────────────────
   Total faces extracted:     12715
   Embedding shape:           (12715, 512)

💾 Results saved to: /content/drive/MyDrive/OTW S.KOM/Embeddings/embeddings_data.pkl

✅ Feature extraction complete!
   Embeddings shape: (12715, 512)
